In [ ]:
!pip install pandas
!pip install sqlalchemy pymysql

In [2]:
# 啟動SERVER前，先RUN這段，推薦演算法
# 傳入參數：標準(陣列)
# 回傳：
def generate_menu(daily_nutrients):
    # 餐別熱量分配權重
    meal_weights = {
        '早餐': 0.2,
        '午餐': 0.3,
        '晚餐': 0.25,
        '飲料': 0.05,
        '點心': 0.1
    }

    # 每餐營養素標準及可接受範圍
    meal_nutrient_limits = {
        meal: {
            nutrient: {
                'standard': daily_nutrients[nutrient] * weight,
                'min': daily_nutrients[nutrient] * weight * 0.8,
                'max': daily_nutrients[nutrient] * weight * 1.2
            }
            for nutrient in daily_nutrients
        }
        for meal, weight in meal_weights.items()
    }

    # 主餐查詢條件
    main_queries = {
        '早餐': """
            SELECT Category_Num, Source, Name, Energy, Protein, Carbohydrate, Total_lipid_fat 
            FROM new 
            WHERE grains > 1 AND Drinks = 0 AND Dessert = 0 AND Category_Num = 4 
            AND Energy >= :energy_min AND Energy <= :energy_max AND Total_lipid_fat <= :fat_max
            ORDER BY RAND() LIMIT 1;
        """,
        '午餐': """
            SELECT Category_Num, Source, Name, Energy, Protein, Carbohydrate, Total_lipid_fat 
            FROM new 
            WHERE grains > 1 AND Drinks = 0 AND Dessert = 0 AND Category_Num NOT IN (4, 8) 
            AND Energy >= :energy_min AND Energy <= :energy_max AND Total_lipid_fat <= :fat_max
            AND Total_lipid_fat >= :fat_min AND Carbohydrate <= :carb_max AND Carbohydrate >= :carb_min
            AND Protein >= :prot_min AND Protein <= :prot_max
            ORDER BY RAND() LIMIT 1;
        """,
        '晚餐': """
            SELECT Category_Num, Source, Name, Energy, Protein, Carbohydrate, Total_lipid_fat 
            FROM new 
            WHERE grains > 1 
            AND Dessert = 0 
            AND Category_Num NOT IN (4, 8)
            AND Drinks = 0
            AND Energy >= :energy_min AND Energy <= :energy_max AND Total_lipid_fat <= :fat_max
            AND Total_lipid_fat >= :fat_min AND Carbohydrate <= :carb_max AND Carbohydrate >= :carb_min
            AND Protein >= :prot_min
            ORDER BY RAND() 
            LIMIT 1;
        """,
        '飲料': 
        """
            SELECT Category_Num, Source, Name, Energy, Protein, Carbohydrate, Total_lipid_fat 
            FROM new 
            WHERE Drinks = 1
            AND Energy >= :energy_min AND Energy <= :energy_max AND Total_lipid_fat >= :fat_min
            ORDER BY RAND() LIMIT 1;
        """,
        '點心': 
        """
            SELECT Category_Num, Source, Name, Energy, Protein, Carbohydrate, Total_lipid_fat 
            FROM new 
            WHERE Dessert = 1 
            AND Energy >= :energy_min AND Energy <= :energy_max AND Protein >= :prot_min AND Total_lipid_fat >= :fat_min
            ORDER BY RAND() LIMIT 1;
        """
    }

    # 副餐查詢條件
    side_queries = {
        '早餐': """
            SELECT Source, Name, Energy, Protein, Carbohydrate, Total_lipid_fat 
            FROM new 
            WHERE grains < 1 AND Dessert = 0 AND Category_Num = 4 
            AND Energy <= :energy_limit 
            ORDER BY RAND() LIMIT 1;
        """,
        '午餐': """
            SELECT Source, Name, Energy, Protein, Carbohydrate, Total_lipid_fat 
            FROM new 
            WHERE grains < 1 AND Dessert = 0 AND Category_Num NOT IN (4) 
            AND Energy <= :energy_limit 
            ORDER BY RAND() LIMIT 1;
        """,
        '晚餐': """
            SELECT Source, Name, Energy, Protein, Carbohydrate, Total_lipid_fat 
            FROM new 
            WHERE grains < 1 AND Dessert = 0 AND Category_Num NOT IN (4) 
            AND Energy <= :energy_limit 
            ORDER BY RAND() LIMIT 1;
        """
    }

    def get_main_dish(meal, main_limits):
        query = text(main_queries[meal])

        # 合併SQL WHERE參數
        params = {'energy_min': main_limits['Energy']['min'], 
                  'energy_max': main_limits['Energy']['max'], 
                  'fat_max': main_limits['Total_lipid_fat']['max'],
                  'fat_min': main_limits['Total_lipid_fat']['min'],
                  'carb_max': main_limits['Carbohydrate']['max'],
                  'carb_min': main_limits['Carbohydrate']['min'],
                  'prot_max': main_limits['Protein']['max'],
                  'prot_min': main_limits['Protein']['min']}
        with engine.connect() as conn:
            result = conn.execute(query, params).mappings().fetchone()
        return result

    def calculate_side_limits(main_totals, nutrient_standard):
        side_limits = {}
        for nutrient, limits in nutrient_standard.items():
            standard = limits['standard']
            side_limits[nutrient] = {
                'min': (standard - main_totals[nutrient]) * 0.9,
                'max': (standard - main_totals[nutrient]) * 1.1
            }
        return side_limits

    def get_side_dish(meal, side_limits):
        query = text(side_queries[meal])
        params = {'energy_limit': side_limits['Energy']['max']}
        with engine.connect() as conn:
            result = conn.execute(query, params).mappings().fetchone()
        return result

    # 推薦組合
    menu = {}
    meal_nutrient_totals = {meal: {key: 0 for key in daily_nutrients} for meal in meal_weights.keys()}
    nutrient_totals = {key: 0 for key in daily_nutrients}

    for meal in main_queries.keys():
        energy_limit = meal_nutrient_limits[meal]['Energy']['standard']
        lipid_fat_limit_max = meal_nutrient_limits[meal]['Total_lipid_fat']['max']
        energy_limit_min = meal_nutrient_limits[meal]['Energy']['min']
        energy_limit_max = meal_nutrient_limits[meal]['Energy']['max']
        main_dish = get_main_dish(meal, meal_nutrient_limits[meal])
        if not main_dish:
            continue
        menu[meal] = [{
            'Type': '主餐',
            'Source': main_dish['Source'],
            'Name': main_dish['Name'],
            'Nutrients': {
                'Energy': main_dish['Energy'],
                'Protein': main_dish['Protein'],
                'Total_lipid_fat': main_dish['Total_lipid_fat'],
                'Carbohydrate': main_dish['Carbohydrate']
            }
        }]
        for nutrient in daily_nutrients:
            meal_nutrient_totals[meal][nutrient] += main_dish[nutrient]
            nutrient_totals[nutrient] += main_dish[nutrient]

        if meal in side_queries:
            side_limits = calculate_side_limits(meal_nutrient_totals[meal], meal_nutrient_limits[meal])
            side_dish = get_side_dish(meal, side_limits)
            if side_dish:
                menu[meal].append({
                    'Type': '副餐',
                    'Source': side_dish['Source'],
                    'Name': side_dish['Name'],
                    'Nutrients': {
                        'Energy': side_dish['Energy'],
                        'Protein': side_dish['Protein'],
                        'Total_lipid_fat': side_dish['Total_lipid_fat'],
                        'Carbohydrate': side_dish['Carbohydrate']
                    }
                })
                for nutrient in daily_nutrients:
                    meal_nutrient_totals[meal][nutrient] += side_dish[nutrient]
                    nutrient_totals[nutrient] += side_dish[nutrient]

    return menu, meal_nutrient_totals, nutrient_totals, meal_nutrient_limits